In [1]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text, interact
from traitlets import link, dlink

In [2]:
for i in range(0,10,2):
    print(i)

0
2
4
6
8


In [3]:
import numpy as np
np.linspace?

In [4]:
"""
Viewing Stanford 3D Scanning Repository bunny model
"""
# Copyright (c) 2014-2015, Enthought, Inc.
# Standard library imports
import os
from os.path import join

### Download the bunny data, if not already on disk ############################
if not os.path.exists('bunny.tar.gz'):
    # Download the data
    try:
        from urllib import urlopen
    except ImportError:
        from urllib.request import urlopen
    print("Downloading bunny model, Please Wait (3MB)")
    opener = urlopen(
                'http://graphics.stanford.edu/pub/3Dscanrep/bunny.tar.gz')
    open('bunny.tar.gz', 'wb').write(opener.read())

# Extract the data
import tarfile
bunny_tar_file = tarfile.open('bunny.tar.gz')
try:
    os.mkdir('bunny_data')
except:
    pass
bunny_tar_file.extractall('bunny_data')
bunny_tar_file.close()

# Path to the bunny ply file
bunny_ply_file = join('bunny_data', 'bunny', 'reconstruction', 'bun_zipper.ply')

In [5]:
f = open(bunny_ply_file)

In [6]:
f = open(bunny_ply_file)
for i in range(12):
    f.readline()
vertices = [np.array(f.readline().split()[:3]).astype(np.float32).tolist() for i in range(35947)]
faces = [np.array(f.readline().split()[1:]).astype(np.float32).tolist() for i in range(69451)]

In [7]:
faces = [np.array(f.readline().split()).astype(np.float32).tolist() for i in range(69451)]

In [8]:
print(faces[0][1:])

[]


In [9]:
vertices

[[-0.037829700857400894, 0.12793999910354614, 0.0044746701605618],
 [-0.044779401272535324, 0.12888699769973755, 0.001904970034956932],
 [-0.06800950318574905, 0.15124399960041046, 0.037195298820734024],
 [-0.0022874099668115377, 0.13015000522136688, 0.02322009950876236],
 [-0.022605400532484055, 0.12667499482631683, 0.007155870087444782],
 [-0.025107799097895622, 0.12592099606990814, 0.006242259871214628],
 [-0.03712090104818344, 0.12744900584220886, 0.0017956000519916415],
 [0.03321300074458122, 0.11269199848175049, 0.02768610045313835],
 [0.038042500615119934, 0.1097550019621849, 0.016168899834156036],
 [-0.025508299469947815, 0.11256799846887589, 0.03667670115828514],
 [-0.024530600756406784, 0.11263599991798401, 0.03734689950942993],
 [0.027403099462389946, 0.1215599998831749, 0.021220799535512924],
 [-0.06289610266685486, 0.15841899812221527, -0.017587099224328995],
 [0.04008129984140396, 0.10420200228691101, 0.022168399766087532],
 [0.04515320062637329, 0.093196801841259, 0.0111

In [17]:
import numpy as np

In [20]:
vsr.Biv(np.pi/4,0,0).exp().quat()

array([ 0.        , -0.        , -0.70710678,  0.70710678])

In [13]:
bunnies = [Mesh(geometry = PlainGeometry(vertices = vertices, faces=faces),
                             material= BasicMaterial(color='cyan'),
               position=[0,0,-0.3])] +\
              [Mesh(geometry = PlainGeometry(vertices = vertices2, faces=faces),
                             material= BasicMaterial(color='yellow'))]

In [ ]:
plane = Mesh(geometry=PlaneGeometry(widthSegments=8,
                                    heightSegments=8), 
             material=BasicMaterial(color='hotpink', 
                                    wireframe=True), 
             quaternion=vsr.Biv(0,0,np.pi/4).exp().quat().tolist())
scene = Scene(children = bunnies + [plane] )
c = PerspectiveCamera(position=[0.5, 0.25, 0.5])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = 'white')
display(renderer)

In [42]:
bunnies[0].material.wireframe = True

In [44]:
plane.material.wireframe = True

In [11]:
import sys
sys.path.append('../build/')
import versor as vsr
from motor_estimation import MotorEstimationSolver

In [12]:
motor0 = vsr.Mot(1,0,0,0,0,0,0,0)
motor1 = vsr.Vec(0.05,0.,0.).trs() * vsr.Rot(vsr.Biv(0,1,0) * np.pi/6.0)
points_a = [vsr.Vec(*v).null() for v in vertices]
points_b = [p.spin(motor1) for p in points_a]
vertices2 = [np.array(p)[:3].tolist() for p in points_b]

In [ ]:
scene = Scene(children=[Mesh(geometry = SphereGeometry(radius=0.001),
                             material= BasicMaterial(color='hotpink'),
                             position=np.array(points_a[i])[:3].tolist()) 
                        for i in range(0,35947,4)] 
              + [Mesh(geometry = SphereGeometry(radius=0.001),
                      material= BasicMaterial(color='yellow'),
                      position=np.array(points_b[i])[:3].tolist()) 
                      for i in range(0,35947,4)])
c = PerspectiveCamera(position=[0.5, 0.25, 0.5])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = 'gray')
display(renderer)

In [69]:
initial_motor = vsr.Mot(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0)
# initial_motor = vsr.Vec(2,3,4).trs() * vsr.Rot(vsr.Biv(0,-1,0) * np.pi/6)
mes = MotorEstimationSolver(initial_motor)
for a, b in zip(points_a, points_b):
    mes.add_point_correspondences_residual_block(a,b)
mes.set_parameterization('BIVECTOR_GENERATOR')
mes.linear_solver_type = 'DENSE_QR'
(estimated_motor, summary_point_1, _) = mes.solve()
print(estimated_motor)
print(summary_point_1['full_report'])

Mot: [ 0.87 6.4e-11 -0.5 -5.7e-11 -0.022 6.1e-12 0.013 5.9e-12 ]

Solver Summary (v 1.11.0-eigen-(3.2.92)-lapack-suitesparse-(4.4.6)-openmp)

                                     Original                  Reduced
Parameter blocks                            1                        1
Parameters                                  8                        8
Effective parameters                        6                        6
Residual blocks                         35947                    35947
Residual                               107841                   107841

Minimizer                        TRUST_REGION

Dense linear algebra library            EIGEN
Trust region strategy     LEVENBERG_MARQUARDT

                                        Given                     Used
Linear solver                        DENSE_QR                 DENSE_QR
Threads                                     1                        1
Linear solver threads                       1                        1

Cost:


In [83]:
def lights_gray():
    return [
        AmbientLight(color='gray'),
#         DirectionalLight(position=[0, 1, 1], color='gray'),
#         DirectionalLight(position=[0, 0, 1], color='gray'),
        DirectionalLight(position=[1, 1, 2], color='gray'),
    ]
lights_gray()

In [84]:
plane = Mesh(geometry=PlaneGeometry(widthSegments=4,
                                    heightSegments=4), 
             material=PhongMaterial(color='hotpink', 
                                    wireframe=False))
sphere = Mesh(geometry=SphereGeometry(radius=0.25),
              material=PhongMaterial(color='gray'),
              position=[0,0,0.25])
sphere.castShadow = True
sphere.receiveShadow = True
plane.receiveShadow = True
ambientlight = AmbientLight(color='white')
# directionallight = DirectionalLight(color='white')
scene = Scene(children = [plane, sphere] + lights_gray() )
c = PerspectiveCamera(position=[0, 0, 2])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = 'black')
display(renderer)

In [116]:
from urllib import urlopen
o = urlopen('https://raw.githubusercontent.com/mrdoob/three.js/master/examples/models/ply/ascii/dolphins_colored.ply')
dolphinply = o.read().split('\n')
print(dolphinply[:14])

['ply', 'format ascii 1.0', 'comment VCGLIB generated', 'element vertex 855', 'property float x', 'property float y', 'property float z', 'property uchar red', 'property uchar green', 'property uchar blue', 'property uchar alpha', 'element face 1689', 'property list uchar int vertex_indices', 'end_header']


In [94]:
def rgb2hex(r, g, b):
    return "#{0:02x}{1:02x}{2:02x}".format(r, g, b)

ValueError: string size must be a multiple of element size

In [113]:
v = [ np.array(v.split()).astype(np.float32) for v in dolphinply[14:]]

In [166]:
vertices = [(v[:3], v[3:-1], v[-1]) 
            for v in [np.array(v.split()).astype(np.float32) 
                      for v in dolphinply[14:14+855]]]
ps = [p.tolist() for p, c, a in vertices]
cs = [rgb2hex(*c.astype(np.int)) for p, c, a in vertices]
faces = [np.array(v.split())[1:].astype(np.int).tolist() for v in dolphinply[14+855:-2]]

In [175]:
dolphin = Mesh(geometry=PlainGeometry(vertices=ps, 
                                       faces=faces,
                                    colors=cs),
               material=BasicMaterial(vertexColors='VertexColors',
                                     wireframe=False))
scene = Scene(children=[dolphin])
c = PerspectiveCamera(position=[3, 5, 4])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = 'gray')
display(renderer)

In [192]:
o = Object3d()
line = Line(geometry=PlainGeometry(vertices=[[0,0,0], [0,1,0]]),
            material=LineBasicMaterial(color='red'))
line.position = [0,-1,0]
head  = Mesh(geometry = CylinderGeometry(radiusTop=0,
                            radiusBottom = 0.5,
                                        height=1,
                                        radiusSegments=5,
                                        heightSegments=1),
             material=BasicMaterial(color='red'),
            position=[0,-0.5,0])
o.children = [head, line]
scene = Scene(children=[o])
c = PerspectiveCamera(position=[3, 5, 4])
renderer = Renderer(camera=c, scene = scene, controls=[TrackballControls(controlling=c)],
                    background = 'gray')
display(renderer)